In [1]:
from pyspark.sql import DataFrame, Row
from functools import reduce
from pyspark.sql.types import *
from pyspark.sql.functions import *
from graphframes import *
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
import uuid

In [2]:
conf=SparkConf()
conf.set("spark.executor.memory", "2g")
conf.set("spark.driver.memory", "12g")
conf.set("spark.cores.max", "6")

sc = SparkContext.getOrCreate(conf)

spark = SQLContext(sc)

23/12/06 20:06:02 WARN Utils: Your hostname, preacher resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface enp4s0)
23/12/06 20:06:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/06 20:06:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/preacher/anaconda3/envs/deltarun/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
match_schema = StructType([
    StructField("pair_id", StringType(), True),
    StructField("sku_uuid_a", StringType(), True),
    StructField("base_source_store", StringType(), True),
    StructField("sku_uuid_b", StringType(), True),
    StructField("comp_source_store", StringType(), True),
    StructField("score", DoubleType(), True),
    StructField("segment", StringType(), True),
    StructField("seller_type", StringType(), True),
    StructField("year", StringType(), True),
    StructField("month", StringType(), True),
    StructField("day", StringType(), True),
    StructField("run_number", StringType(), True),
    StructField("source", StringType(), True),
])


In [4]:
upc_match = [
    ["0000346280418<>203359001<>homedepot<>homedepot_346280418<>g4103325<>bcg<>bcg", "0000346280418<>203359001<>homedepot<>homedepot","homedepot<>homedepot","346280418<>g4103325<>bcg<>bcg", "bcg<>bcg",1.0,"pets","1P","2023","07","17", "01","UPC"],
    ["0000346318654<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions","0000346318654<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","UPC"],
    ["0000346378252<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions", "0000346378252<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions","hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","UPC"],
    ["0000346383867<>100670057<>homedepot<>homedepot_000346383867<>967493<>hdsupplysolutions<>hdsupplysolutions", "0000346383867<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","000346383867<>967493<>hdsupplysolutions<>hdsupplysolutions","hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","UPC"],
    ["00003463186541<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions", "00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions","hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","UPC"], # > 0.05
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000432<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000432<>147664<>chewy<>chewy", "chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","UPC"], # appended one old not present
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy", "chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","UPC"], # appended
    ["00003463186541<>100341666<>homedepot<>homedepot_0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","UPC"], # > 0.05 + added
]

In [5]:
mpn_match = [
    ["0000346280418<>203359001<>homedepot<>homedepot_346280418<>g4103325<>bcg<>bcg","0000346280418<>203359001<>homedepot<>homedepot", "homedepot<>homedepot","346280418<>g4103325<>bcg<>bcg", "bcg<>bcg",1.0,"pets","1P","2023","07","17", "01","MPN"],
    ["0000346318654<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions","0000346318654<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","MPN"],
    ["0000346383843<>202080350<>homedepot<>homedepot_346383843<>g1969694<>bcg<>bcg","0000346383843<>202080350<>homedepot<>homedepot", "homedepot<>homedepot","346383843<>g1969694<>bcg<>bcg", "bcg<>bcg",1.0,"pets","1P","2023","07","17", "01","MPN"],
    ["0000346392418<>204823270<>homedepot<>homedepot_000346392418<>453560<>hdsupplysolutions<>hdsupplysolutions","0000346392418<>204823270<>homedepot<>homedepot", "homedepot<>homedepot","000346392418<>453560<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","MPN"],
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions","00003463782521<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","MPN"], # < 0.05
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions","00003463782521<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","MPN"], # < 0.05 + added
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy","0000000000001<>53409<>petcarerx<>petcarerx", "petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy", "chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","MPN"], # appended
]

In [6]:
ml_data = [
    ["0000346280418<>203359001<>homedepot<>homedepot_346280418<>g4103325<>bcg<>bcg","0000346280418<>203359001<>homedepot<>homedepot", "homedepot<>homedepot","346280418<>g4103325<>bcg<>bcg", "bcg<>bcg",0.987,"pets","1P","2023","07","17", "01","ML"],
    ["0000346378252<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions","0000346378252<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",0.876,"pets","1P","2023","07","17", "01","ML"],
    ["0000346383843<>202080350<>homedepot<>homedepot_346383843<>g1969694<>bcg<>bcg","0000346383843<>202080350<>homedepot<>homedepot", "homedepot<>homedepot","346383843<>g1969694<>bcg<>bcg", "bcg<>bcg",0.567,"pets","1P","2023","07","17", "01","ML"],
    ["000000000000<>53409<>petcarerx<>petcarerx_000000000000<>53417<>petcarerx<>petcarerx","000000000000<>53409<>petcarerx<>petcarerx", "petcarerx<>petcarerx","000000000000<>53417<>petcarerx<>petcarerx", "petcarerx<>petcarerx",0.8737,"pets","1P","2023","07","17", "01","ML"],
    ["00003463186541<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",0.8737,"pets","1P","2023","07","17", "01","ML"], # > 0.05
    ["00003463186541<>100341666<>homedepot<>homedepot_0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",0.8737,"pets","1P","2023","07","17", "01","ML"], # > 0.05 + added
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions","00003463782521<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",0.943,"pets","1P","2023","07","17", "01","ML"], # < 0.05
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions","00003463782521<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",0.943,"pets","1P","2023","07","17", "01","ML"], # < 0.05 + added
]

In [7]:
transitive_data = [
    ["0000346318654<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions", "0000346318654<>100341666<>homedepot<>homedepot","homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"],
    ["0000346378252<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions", "0000346378252<>203370169<>homedepot<>homedepot","homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"],
    ["0000346383843<>202080350<>homedepot<>homedepot_346383843<>g1969694<>bcg<>bcg", "0000346383843<>202080350<>homedepot<>homedepot","homedepot<>homedepot","346383843<>g1969694<>bcg<>bcg", "bcg<>bcg",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"],
    ["000000000000<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy", "000000000000<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy", "chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"],
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions", "00003463782521<>203370169<>homedepot<>homedepot","homedepot<>homedepot","000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"], # < 0.05 + added
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000123<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000123<>147664<>chewy<>chewy", "chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"], # added
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000432<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000432<>147664<>chewy<>chewy", "chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"], # appended one old not present
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy", "chewy<>chewy",1.0,"pets","1P","2023","07","17", "01","TRANSITIVE"], # appended
]

In [8]:
mdw_schema = StructType([
    StructField("pair_id", StringType(), True),
    StructField("base_sku_uuid", StringType(), True),
    StructField("base_source_store", StringType(), True),
    StructField("comp_sku_uuid", StringType(), True),
    StructField("comp_source_store", StringType(), True),
    StructField("segment", StringType(), True),
    StructField("match_source_score_map", MapType(StringType(), DoubleType()), True),
    StructField("aggregated_score", DoubleType(), True),
    StructField("seller_type", StringType(), True),
])

In [9]:
mdw_data = [
    ["00003463186541<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions", "00003463186541<>100341666<>homedepot<>homedepot","homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions","hdsupplysolutions<>hdsupplysolutions","pets",{"ML": 0.745, "UPC": 1.0},1.745,"1P"], # > 0.05
    ["00003463186541<>100341666<>homedepot<>homedepot_0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions", "00003463186541<>100341666<>homedepot<>homedepot","homedepot<>homedepot","0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions","hdsupplysolutions<>hdsupplysolutions","pets",{"ML": 0.745, "MPN": 1.0},1.745,"1P"], # > 0.05 + added
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions", "00003463782521<>203370169<>homedepot<>homedepot","homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions","hdsupplysolutions<>hdsupplysolutions","pets",{"ML": 0.945, "MPN": 1.0},1.945,"1P"], # < 0.05
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions", "00003463782521<>203370169<>homedepot<>homedepot","homedepot<>homedepot","000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions","hdsupplysolutions<>hdsupplysolutions","pets",{"ML": 0.945, "MPN": 1.0},1.945,"1P"], # < 0.05 + added
    ["00003463838431<>202080350<>homedepot<>homedepot_346383843<>g1969694<>bcg<>bcg", "00003463838431<>202080350<>homedepot<>homedepot","homedepot<>homedepot","346383843<>g1969694<>bcg<>bcg","bcg<>bcg","pets", {"UPC": 1.0, "MPN": 1.0},2.0,"1P"], # not added
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy","chewy<>chewy","pets", {"UPC": 1.0, "MPN": 1.0},2.0,"1P"], # appended
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000432<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000432<>147664<>chewy<>chewy","chewy<>chewy","pets", {"UPC": 1.0, "MPN": 1.0},2.0,"1P"], # appended one old not present
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000123<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000123<>147664<>chewy<>chewy","chewy<>chewy","pets", {"UPC": 1.0, "MPN": 1.0},2.0,"1P"], # added
]

In [23]:
New = [
["0000346280418<>203359001<>homedepot<>homedepot_346280418<>g4103325<>bcg<>bcg", "0000346280418<>203359001<>homedepot<>homedepot","homedepot<>homedepot","346280418<>g4103325<>bcg<>bcg", "bcg<>bcg",{"UPC": 1.0, "MPN":1.0, "ML": 0.987},"pets","1P"],
["0000346318654<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions","hdsupplysolutions<>hdsupplysolutions", "0000346318654<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions",{"UPC": 1.0, "MPN":1.0, "TRANSITIVE": 1.0},"pets","1P"],
["0000346378252<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions","hdsupplysolutions<>hdsupplysolutions", "0000346378252<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions",{"UPC": 1.0, "TRANSITIVE":1.0, "ML" : 0.876},"pets","1P"],
["0000346383867<>100670057<>homedepot<>homedepot_000346383867<>967493<>hdsupplysolutions<>hdsupplysolutions","hdsupplysolutions<>hdsupplysolutions", "0000346383867<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","000346383867<>967493<>hdsupplysolutions<>hdsupplysolutions",{"UPC": 1.0},"pets","1P"],
["0000346383843<>202080350<>homedepot<>homedepot_346383843<>g1969694<>bcg<>bcg","0000346383843<>202080350<>homedepot<>homedepot", "homedepot<>homedepot","346383843<>g1969694<>bcg<>bcg", "bcg<>bcg",{"MPN": 1.0, "TRANSITIVE":1.0, "ML" : 0.567},"pets","1P"],
["0000346392418<>204823270<>homedepot<>homedepot_000346392418<>453560<>hdsupplysolutions<>hdsupplysolutions","0000346392418<>204823270<>homedepot<>homedepot", "homedepot<>homedepot","000346392418<>453560<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",{"MPN": 1.0},"pets","1P"],
["000000000000<>53409<>petcarerx<>petcarerx_000000000000<>53417<>petcarerx<>petcarerx","000000000000<>53409<>petcarerx<>petcarerx", "petcarerx<>petcarerx","000000000000<>53417<>petcarerx<>petcarerx", "petcarerx<>petcarerx",{"ML": 0.8737},"pets","1P"],
["000000000000<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy", "000000000000<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy", "chewy<>chewy",{"TRANSITIVE":1.0},"pets","1P"],
]
update = [
["00003463186541<>100341666<>homedepot<>homedepot_000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions","hdsupplysolutions<>hdsupplysolutions", "00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","000346318654<>109589<>hdsupplysolutions<>hdsupplysolutions",{"UPC": 1.0, "ML": 0.8737},"pets","1P"], # > 0.05
["0000000000001<>53409<>petcarerx<>petcarerx_00000432<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000432<>147664<>chewy<>chewy", "chewy<>chewy",{"UPC": 1.0, "TRANSITIVE": 1.0, "MPN": 1.0},"pets","1P"], # appended one old not present
["0000000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy", "chewy<>chewy",{"UPC": 1.0, "MPN":1.0, "TRANSITIVE": 1.0},"pets","1P"], # appended
["00003463186541<>100341666<>homedepot<>homedepot_0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","0003463186541<>109589<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",{"UPC": 1.0, "ML": 0.8737, "MPN": 1.0},"pets","1P"], # > 0.05 + added
["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions", "00003463782521<>203370169<>homedepot<>homedepot","homedepot<>homedepot","000346378252<>9674462<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",{"MPN": 1.0, "ML": 0.945, "TRANSITIVE": 1.0},"pets","1P"], # < 0.05 + added
["0000000000001<>53409<>petcarerx<>petcarerx_00000123<>147664<>chewy<>chewy", "0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000123<>147664<>chewy<>chewy", "chewy<>chewy",{"TRANSITIVE":1.0,"UPC": 1.0, "MPN": 1.0}, "pets","1P"], # added
]
rejected = [
    ["00003463782521<>203370169<>homedepot<>homedepot_000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions","00003463782521<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","000346378252<>967446<>hdsupplysolutions<>hdsupplysolutions", "hdsupplysolutions<>hdsupplysolutions",{"MPN": 1.0, "ML": 0.945},"pets","1P"] # < 0.05
]


In [11]:
mdw = spark.createDataFrame(mdw_data, schema=mdw_schema)

In [12]:
ml_match_suggestion = df = spark.createDataFrame(ml_data, schema=match_schema)
upc_match_suggestion = spark.createDataFrame(upc_match, schema=match_schema)
mpn_match_suggestion = spark.createDataFrame(mpn_match, schema=match_schema)
transitive_match_suggestion = spark.createDataFrame(transitive_data, schema=match_schema)


In [13]:
from aggregator import Aggregator
aggregator = Aggregator(ml_match_suggestion,
upc_match_suggestion,
mpn_match_suggestion,
transitive_match_suggestion,
mdw)

Combining dfs


In [14]:
merged_matches = aggregator.merged_match_suggestion
mdw = aggregator.mdw

In [15]:
merged_matches.printSchema()
mdw.printSchema()

root
 |-- pair_id: string (nullable = true)
 |-- sku_uuid_a: string (nullable = true)
 |-- new_base_source_store: string (nullable = true)
 |-- sku_uuid_b: string (nullable = true)
 |-- new_comp_source_store: string (nullable = true)
 |-- new_score: double (nullable = true)
 |-- new_segment: string (nullable = true)
 |-- new_seller_type: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- run_number: string (nullable = true)
 |-- source: string (nullable = true)

root
 |-- pair_id: string (nullable = true)
 |-- base_sku_uuid: string (nullable = true)
 |-- base_source_store: string (nullable = true)
 |-- comp_sku_uuid: string (nullable = true)
 |-- comp_source_store: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- match_source_score_map: map (nullable = true)
 |    |-- key: string
 |    |-- value: double (valueContainsNull = true)
 |-- aggregated_score: double (nullable = true)
 |-

In [16]:
merged_matches = aggregator.aggregate_match_suggestion_and_mdw(mdw, merged_matches)

+--------------------+------+--------------------+---------------------+--------------------+---------------------+---------+-----------+---------------+----+-----+----+----------+--------------------+---------------------+--------------------+---------------------+-----------+----------------------+----------------+---------------+---------+
|             pair_id|source|          sku_uuid_a|new_base_source_store|          sku_uuid_b|new_comp_source_store|new_score|new_segment|new_seller_type|year|month| day|run_number|       base_sku_uuid|old_base_source_store|       comp_sku_uuid|old_comp_source_store|old_segment|match_source_score_map|aggregated_score|old_seller_type|old_score|
+--------------------+------+--------------------+---------------------+--------------------+---------------------+---------+-----------+---------------+----+-----+----+----------+--------------------+---------------------+--------------------+---------------------+-----------+----------------------+---------

In [17]:
merged_matches.show()

+--------------------+----------+---------+----+-----+----+----------+--------------------+--------------------+----------------------+----------------+---------+------+-------+-----------+--------------------+--------------------+------------+
|             pair_id|    source|new_score|year|month| day|run_number|       base_sku_uuid|       comp_sku_uuid|match_source_score_map|aggregated_score|old_score| score|segment|seller_type|   base_source_store|   comp_source_store|match_status|
+--------------------+----------+---------+----+-----+----+----------+--------------------+--------------------+----------------------+----------------+---------+------+-------+-----------+--------------------+--------------------+------------+
|0000000000001<>53...|       MPN|      1.0|2023|   07|  17|        01|0000000000001<>53...|00000000<>147664<...|  {UPC -> 1.0, MPN ...|             2.0|      1.0|   1.0|   pets|         1P|petcarerx<>petcarerx|        chewy<>chewy| not_updated|
|0000000000001<>53..

In [18]:
updated_matches_suggestion, new_matches_suggestion = aggregator.fetch_updated_and_new_matches_suggestions(merged_matches)

In [19]:
updated_matches_suggestion.show(truncate =False, n = 100)
new_matches_suggestion.show(truncate =False, n = 100)

+-----------------------------------------------------------------------------------------------------------+-------+-----------------------------------------------+-----------------------------------------------------------+--------------------+------------------------------------+-----------+--------------------------------------------+---------------------------+----------------+
|pair_id                                                                                                    |segment|base_sku_uuid                                  |comp_sku_uuid                                              |base_source_store   |comp_source_store                   |seller_type|match_source_score_map                      |match_status               |aggregated_score|
+-----------------------------------------------------------------------------------------------------------+-------+-----------------------------------------------+-----------------------------------------------------------+---

In [20]:
new_matches_suggestion = aggregator.convert_schema_for_new_match_suggestion(new_matches_suggestion)
new_matches_suggestion.show()

+--------------------+-------+--------------------+--------------------+----------------------+----------------+--------------------+--------------------+------------+----------+-------------------+--------------+-----------------+----------------------+----------------------+-----------------+--------------------+-------------------------+----------------------+-----------------+--------------------+-------------------------+---------+
|             pair_id|segment|       base_sku_uuid|       comp_sku_uuid|match_source_score_map|aggregated_score|        created_date|          created_by|updated_date|updated_by|bungee_audit_status|bungee_auditor|bungee_audit_date|bungee_auditor_comment|client_audit_status_l1|client_auditor_l1|client_audit_date_l1|client_auditor_l1_comment|client_audit_status_l2|client_auditor_l2|client_audit_date_l2|client_auditor_l2_comment|misc_info|
+--------------------+-------+--------------------+--------------------+----------------------+----------------+------

In [1]:
un_audited single word and status in capital
bungee_auditor -> glue job name


SyntaxError: invalid syntax (3622547193.py, line 1)

In [ ]:
for manual upload we have to updated the map with source as MANUAL and updated score as well